In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
## importing the data
housing = fetch_california_housing()

In [4]:
## splitting the data into train, validation and test data
x_train_full, x_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

In [5]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full)

In [6]:
## scaling the data
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_valid = scaler.transform(x_valid)
x_test = scaler.transform(x_test)

In [7]:
import tensorflow as tf
from tensorflow import keras

2022-10-02 11:41:53.369524: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-02 11:41:53.369555: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [8]:
def build_model(n_hidden,n_neuron,learning_rate,activation,input_shape):

  ## creating a model
  model = keras.models.Sequential()

  ## adding a input layer
  model.add(keras.layers.InputLayer(input_shape=input_shape))

  ## adding hidden layers
  for layer in range(n_hidden):
    model.add(keras.layers.Dense(units=n_neuron, activation=activation))
  
  ## adding output layer
  model.add(keras.layers.Dense(1))

  ## compiling the model
  model.compile(loss='mse',optimizer='adam')

  ## returning the model
  return model

In [9]:
## creating the callbacks for the model
modelcheckpoint_cb = keras.callbacks.ModelCheckpoint('my_nn_model.h5',save_best_only=True)
earlystopping_cb = keras.callbacks.EarlyStopping(patience=10)

In [10]:
pip install scikeras

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
## applying the wrapper around the keras model
from scikeras.wrappers import KerasRegressor
keras_regressor = KerasRegressor(build_model,n_hidden=2,n_neuron=30,learning_rate=3e-3,activation='relu',input_shape=[8])

In [12]:
## creating a parameter dictionary for the tuning
grid_params = dict()
grid_params['n_hidden'] = [0,1,2]
grid_params['n_neuron'] = [5,10,15,20,25,30]
grid_params['learning_rate'] = [2e-3]
grid_params['activation'] = ['relu']

print(grid_params)

{'n_hidden': [0, 1, 2], 'n_neuron': [5, 10, 15, 20, 25, 30], 'learning_rate': [0.002], 'activation': ['relu']}


In [13]:
## creating grid search object
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=keras_regressor, param_grid=grid_params, cv=5)

In [14]:
## training the model using the training data
grid.fit(x_train,y_train, epochs=20, validation_data=(x_valid,y_valid), callbacks=[modelcheckpoint_cb, earlystopping_cb])

Epoch 1/20


2022-10-02 11:41:57.360925: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-02 11:41:57.360961: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-02 11:41:57.360980: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (7d73ea5c522c): /proc/driver/nvidia/version does not exist
2022-10-02 11:41:57.361226: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


291/291 [==============================] - 1s 1ms/step - loss: 7.5713 - val_loss: 8.0743
Epoch 2/20
291/291 [==============================] - 0s 864us/step - loss: 5.5171 - val_loss: 6.3757
Epoch 3/20
291/291 [==============================] - 0s 898us/step - loss: 4.0954 - val_loss: 5.1369
Epoch 4/20
291/291 [==============================] - 0s 899us/step - loss: 3.0602 - val_loss: 4.2091
Epoch 5/20
291/291 [==============================] - 0s 901us/step - loss: 2.3030 - val_loss: 3.5022
Epoch 6/20
291/291 [==============================] - 0s 927us/step - loss: 1.7584 - val_loss: 2.9749
Epoch 7/20
291/291 [==============================] - 0s 921us/step - loss: 1.3768 - val_loss: 2.5761
Epoch 8/20
291/291 [==============================] - 0s 901us/step - loss: 1.1177 - val_loss: 2.3398
Epoch 9/20
291/291 [==============================] - 0s 918us/step - loss: 0.9448 - val_loss: 2.0592
Epoch 10/20
291/291 [==============================] - 0s 884us/step - loss: 0.8331 - val_loss:

GridSearchCV(cv=5,
             estimator=KerasRegressor(activation='relu', input_shape=[8], learning_rate=0.003, model=<function build_model at 0x7f24e878c670>, n_hidden=2, n_neuron=30),
             param_grid={'activation': ['relu'], 'learning_rate': [0.002],
                         'n_hidden': [0, 1, 2],
                         'n_neuron': [5, 10, 15, 20, 25, 30]})

In [15]:
mse_test = grid.score(x_test,y_test)

162/162 [==============================] - 0s 586us/step


In [16]:
mse_test

0.770294529946241